# Generador de logins

Este simulador de logins genera una pequeña tabla de usuarios y empezará a enviar eventos de login y logoff a Kafka. Además hemos escrito una pequeña función de hash que envíe cada mensaje a una partición diferente en función del ID del usuario.

In [ ]:
import uuid
import random
import json
from confluent_kafka import Producer
import time

from ejercicios.pokemons import SEED, TOPIC_USERS, MEAN_LOGIN, NUM_USERS

user_names = [
    'Luis',
    'Jose',
    'Maria',
    'Laura',
    'Fran',
    'Dani',
    'David',
    'Cris',
    'Oscar',
    'Virginia'
]

users = {}
for u in range(NUM_USERS):
    i = str(uuid.uuid4())
    name = user_names[u % len(user_names)]
    age = random.randint(18, 99)
    users[i] = ({'id': i, 'name': name, 'age': age, 'logged': False})

Esta función de hash, extremadamente básica, genera un número de partición a partir de una clave (el ID de usuario). Para una clave y un número de particiones el resultado es siempre el mismo. Se podría usar como balanceo de carga, asumiendo que los IDs son aleatorios. Otros métodos para repartir la carga podría ser una elección realmente aleatoria (es decir, no a partir de una key) o por round robin.

In [ ]:
def choose_partition(num_partitions, key):
    if num_partitions == 1 or key is None:
        return 0
    else:
        try:
            as_num = int(key[0], base=16)
            return as_num % num_partitions
        except:
            return 0

# Imprimimos algunas pruebas para comprobar que funciona
print(choose_partition(1, str(uuid.uuid4())))
print(choose_partition(1, None))

for i in range(10):
    u = str(uuid.uuid4())
    print("{}: {}".format(u, choose_partition(2, u)))

print()
for i in range(10):
    u = str(uuid.uuid4())
    print("{}: {}".format(u, choose_partition(3, u)))


In [ ]:
p = Producer({'bootstrap.servers': 'localhost:9092'})

def delivery_report(err, msg):
    """ Called once for each message produced to indicate delivery result.
        Triggered by poll() or flush(). """
    if err is not None:
        print('Message delivery failed: {}'.format(err))
    else:
        pass
        # print('Message delivered to {} [{}]'.format(msg.topic(), msg.partition()))

while True:
    # Trigger any available delivery report callbacks from previous produce() calls
    p.poll(0)
    topic_list = p.list_topics(TOPIC_USERS)
    topic_metadata = topic_list.topics[TOPIC_USERS]
    num_partitions = len(topic_metadata.partitions.keys())
    
    user_id = random.choice(list(users.keys()))
    partition_num = choose_partition(num_partitions, user_id)

    if users[user_id]['logged']:
        print("Logging {} OFF on {}".format(user_id, partition_num))
        users[user_id]['logged'] = False
        p.produce(TOPIC_USERS,
                  key=user_id,
                  partition=partition_num,
                  value=None,
                  callback=delivery_report)
    else:
        print("Logging {} IN on {}".format(user_id, partition_num))
        users[user_id]['logged'] = True
        p.produce(TOPIC_USERS,
                  key=user_id,
                  partition=partition_num,
                  value=json.dumps(users[user_id]).encode('utf-8'),
                  callback=delivery_report)

    time.sleep(abs(random.gauss(MEAN_LOGIN, 3)))

# Wait for any outstanding messages to be delivered and delivery report
# callbacks to be triggered.
p.flush()
